# **Generating Shakespearean Text Using a Character RNN**

## **`"Windowing"`**: splitting a sequence into batches of shuffled windows

For example, let's split the sequence 0 to 14 into windows of length 5, each shifted by 2 (e.g.,`[0, 1, 2, 3, 4]`, `[2, 3, 4, 5, 6]`, etc.), then shuffle them, and split them into inputs (the first 4 steps) and targets (the last 4 steps) (e.g., `[2, 3, 4, 5, 6]` would be split into `[[2, 3, 4, 5], [3, 4, 5, 6]]`), then create batches of 3 such input/target pairs:


El método `window()` crea un conjunto de datos que contiene ventanas, cada una de las
cuales también se representa como un conjunto de datos. Es un conjunto de datos anidado(**nested datase**),
análogo a una lista de listas. Esto es útil cuando desea transformar cada ventana llamando a
sus métodos de conjunto de datos (por ejemplo, para mezclarlos o procesarlos por lotes).

Al llamar al método `flat_map()`: convierte un conjunto de datos anidado
en un conjunto de datos plano (uno que no contiene conjuntos de datos). Además toma una funcion como argumento que le permite transformar cada conjunto de datos en el conjunto de datos anidado antes de aplanarlo

In [46]:
import tensorflow as tf
import numpy as np

np.random.seed(42)
tf.random.set_seed(42)

n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))

# obtener una sequencia de tamaño n_steps
dataset = dataset.window(n_steps, shift=2, drop_remainder=True)

# Tenga en cuenta que llamamos a batch(n_steps) en cada ventana: dado que todas las
# ventanas tienen exactamente esa longitud, obtendremos un solo tensor para cada una de ellas
dataset = dataset.flat_map(lambda window: window.batch(n_steps))

# Shuffle and get X and Y
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))

# agrupar las ventanas en batch
dataset = dataset.batch(3).prefetch(1)

# mostrar el dataset
for index, (X_batch, Y_batch) in enumerate(dataset):
    print("_" * 20, "Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("=" * 5, "\nY_batch")
    print(Y_batch.numpy())


____________________ Batch 0 
X_batch
[[6 7 8 9]
 [2 3 4 5]
 [4 5 6 7]]
===== 
Y_batch
[[ 7  8  9 10]
 [ 3  4  5  6]
 [ 5  6  7  8]]
____________________ Batch 1 
X_batch
[[ 0  1  2  3]
 [ 8  9 10 11]
 [10 11 12 13]]
===== 
Y_batch
[[ 1  2  3  4]
 [ 9 10 11 12]
 [11 12 13 14]]


## Loading the Data and Preparing the Dataset

In [47]:
from tensorflow import keras
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()


In [48]:
print(shakespeare_text[:148])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [49]:
"".join(sorted(set(shakespeare_text.lower())))


"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

 Encode every character as an integer. One option is to create
a custom **preprocessing layer**, as` tf.keras.layers.TextVectorization.` But in this case, it
will be simpler to use Keras’s Tokenizer class

In [50]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)


In [51]:
tokenizer.texts_to_sequences(["First and"])


[[20, 6, 9, 8, 3, 1, 5, 10, 13]]

In [52]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3, 1, 5, 10, 13]])


['f i r s t   a n d']

## Building the dataset and **windowing**

In [53]:
max_id = len(tokenizer.word_index)  # number of distinct characters
dataset_size = tokenizer.document_count  # total number of characters

# print(max_id, dataset_size)

# restamos uno pq  tokenizer comienza los inidices en uno
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])


In [54]:
n_steps = 100
window_length = n_steps + 1  # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

dataset = dataset.flat_map(lambda window: window.batch(window_length))


batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

Como se discutió en el *'13_3preprocessing_data.ipynb'*, las características de entrada categóricas generalmente deben
codificarse, generalmente como vectores one hot o como embeddings(incrustaciones). Aquí, codificaremos
cada carácter utilizando un vector único porque hay bastantes pocos caracteres distintos (solo 39)

In [55]:
# obtener el vector X como un vector one_hot
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id),
                              Y_batch))

dataset = dataset.prefetch(1)


In [56]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)


(32, 100, 39) (32, 100)


## Creating and Training the Model

**Warning**: the following code may take up to 24 hours to run, depending on your hardware. If you use a GPU, it may take just 1 or 2 hours, or less.

**Note**: the `GRU` class will only use the GPU (if you have one) when using the default values for the following arguments: `activation`, `recurrent_activation`, `recurrent_dropout`, `unroll`, `use_bias` and `reset_after`. This is why I commented out `recurrent_dropout=0.2` (compared to the book).

In [59]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     # dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     # dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=2)


## Using the Model to Generate Text

In [61]:
def preprocess(texts):
    """ Procesar el texto al mismo formato de entrada de la RNN
    """
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)


In [ ]:
X_new = preprocess(["How are yo"])

# Y_pred = model.predict_classes(X_new) # deprecated
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]  # 1st sentence, last char


'u'

In [ ]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()


array([[0, 1, 0, 2, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 2, 1, 0, 2, 1,
        0, 1, 2, 1, 1, 1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2]])

Para generar texto
nuevo usando el modelo Char RNN, podríamos alimentarlo con algo de texto, hacer que el
modelo prediga la próxima letra más probable, agregarla al final del texto y luego darle el texto
extendido al modelo para adivinar la siguiente letra, y así sucesivamente.
 
Pero en la práctica esto a menudo conduce a que las mismas palabras se repitan una y otra vez. En su lugar, podemos elegir el siguiente carácter al azar, con una probabilidad igual a la probabilidad 
estimada, usando la función TensorFlow **tf.random.categorical()**. Esto generará un texto más diverso e
interesante. La función categorical() muestra índices de clase aleatorios, dadas las **class log probabilities (logits)**. Para tener más control sobre la diversidad del texto
generado, podemos dividir los logits por un número llamado temperatura , que podemos
ajustar como queramos: una temperatura cercana a 0 favorecerá los caracteres de alta probabilidad,
mientras que una temperatura muy alta dara a todos los caracterers igual probabilidad

In [ ]:
tf.random.set_seed(42)


def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]


next_char("How are yo", temperature=1)


'u'

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text


In [ ]:
tf.random.set_seed(42)

print(complete_text("t", temperature=0.2))


the belly the charges of the other words
and belly 


In [ ]:
print(complete_text("t", temperature=1))


thing! they know't.

biondello:
for you are the own


In [ ]:
print(complete_text("t", temperature=2))


th no cyty
use ffor was firive this toighingaber; b


## Stateful RNN

De hecho, si tuviéramos que llamar al `batch(32)`, entonces 32 ventanas consecutivas se colocarían en el
mismo lote, y el siguiente lote no continuaría cada una de estas ventanas donde
las dejó. El primer lote contienen las ventanas 1 a 32 y el segundo lote contendría las ventanas 33 a 64, por lo que
si considera, digamos, la primera ventana de cada lote (es decir, las ventanas 1 y 33), puede
ver que no son consecutivas. La solución más simple a este problema es simplemente usar
"lotes" que contengan una sola ventana

In [63]:
tf.random.set_seed(42)

dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

# notar shift=n_steps para que no haya superposicion
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)

dataset = dataset.flat_map(lambda window: window.batch(window_length))


dataset = dataset.batch(1)

dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)


Batching is harder, but it is not impossible. For example, we could chop
Shakespeare’s text into 32 texts of equal length, create one dataset of
consecutive input sequences for each of them, and finally use
`tf.train.Dataset.zip(datasets).map(lambda *windows:tf.stack(windows))` to create proper consecutive batches, where the n
input sequence in a batch starts off exactly where the n input sequence
ended in the previous batch. For example 3 batch with four elements look loke this: 

 `{1,4,7,10} {2,5,8,11}, {3,6,9,12}`

In [64]:
batch_size = 32

# dividir el texto en 32 arreglos de texto iguales
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []

# por cada uno de los 32 datasets aplicarle Windowing 
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
    
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))

dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)


We need to set `stateful=True `when creating every recurrent layer. Second, the stateful RNN needs to
know the batch size (since it will preserve a state for each input sequence
in the batch)

In [65]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     # dropout=0.2, recurrent_dropout=0.2,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     # dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])


Al final de cada epoach, necesitamos restablecer los estados antes de volver al principio del texto. Para
esto, podemos usar una **Callback**

In [66]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()


model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])


### Clone the model 
Después de entrenar este modelo, solo será posible usarlo para hacer predicciones para lotes del mismo tamaño
que se usaron durante el entrenamiento. Para evitar esta restricción, cree un modelo sin estado idéntico y
copie los pesos del modelo con estado en este modelo

In [ ]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])


To set the weights, we first need to build the model (so the weights get created):

In [ ]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

stateless_model.set_weights(model.get_weights())
model = stateless_model


In [ ]:
tf.random.set_seed(42)

print(complete_text("t"))


tor:
in the negver up how it thou like him;
when it
